# Vessel Seg 全流程示例（ASOCA/RCF/Shape）

按顺序演示：环境检查 → 数据准备 → 形状特征 → 分段 → RCF 推理（PNG→NIfTI）。命令均在仓库根目录执行。


In [1]:
import sys, os, json, subprocess, textwrap
from pathlib import Path
print("cwd:", Path.cwd())
print("Python:", sys.version)
print("vesselfm env?", os.environ.get("CONDA_DEFAULT_ENV"))

cwd: /mnt/c/users/chenyihao/Documents/GitHub/vessel_seg
Python: 3.9.23 (main, Jun  5 2025, 13:40:20) 
[GCC 11.2.0]
vesselfm env? vesselfm


## 1. 形状特征与分段（以 Normal_1 为例）


In [2]:
%%bash
# 提取中心线 + 极坐标截面特征
/home/chenyihao/miniconda3/envs/vesselfm/bin/python -m vessel_seg.shape extract \
  --seg ASOCA2020/Normal/Annotations_nii/Normal_1.nii.gz \
  --out outputs/Normal_1_features


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/chenyihao/miniconda3/envs/vesselfm/lib/python3.9/runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/home/chenyihao/miniconda3/envs/vesselfm/lib/python3.9/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/mnt/c/users/chenyihao/Documents/GitHub/vessel_seg/vessel_seg/__init__.py", line 12, in <module>
    from .shape import (
  File "/mnt/c/users/chenyihao/Documents/GitHub/vessel_seg/vessel_seg/shape.py", line 19, 

{
  "branch_count": 7,
  "branches": [
    {
      "name": "Major_00",
      "length_mm": 94.13656830029274,
      "confidence": 0.9373611211776733,
      "feature_file": "Major_00.npz"
    },
    {
      "name": "Major_01",
      "length_mm": 54.735008754385156,
      "confidence": 0.41430556774139404,
      "feature_file": "Major_01.npz"
    },
    {
      "name": "Major_02",
      "length_mm": 46.94484207308679,
      "confidence": 1.0,
      "feature_file": "Major_02.npz"
    },
    {
      "name": "Branch_03",
      "length_mm": 37.217573225208476,
      "confidence": 1.0,
      "feature_file": "Branch_03.npz"
    },
    {
      "name": "Branch_04",
      "length_mm": 30.470962889120624,
      "confidence": 1.0,
      "feature_file": "Branch_04.npz"
    },
    {
      "name": "Branch_05",
      "length_mm": 20.792064150492216,
      "confidence": 0.37680548429489136,
      "feature_file": "Branch_05.npz"
    },
    {
      "name": "Branch_06",
      "length_mm": 15.430399556033043

In [3]:
%%bash
# 按弧长 10mm 切段，记录相邻段夹角
/home/chenyihao/miniconda3/envs/vesselfm/bin/python scripts/split_branch_segments.py \
  --features outputs/Normal_1_features \
  --out outputs/Normal_1_segments \
  --segment-length 10 \
  --min-segment-length 2

/home/chenyihao/miniconda3/envs/vesselfm/bin/python: can't open file '/mnt/c/users/chenyihao/Documents/GitHub/vessel_seg/scripts/split_branch_segments.py': [Errno 2] No such file or directory


CalledProcessError: Command 'b'# \xe6\x8c\x89\xe5\xbc\xa7\xe9\x95\xbf 10mm \xe5\x88\x87\xe6\xae\xb5\xef\xbc\x8c\xe8\xae\xb0\xe5\xbd\x95\xe7\x9b\xb8\xe9\x82\xbb\xe6\xae\xb5\xe5\xa4\xb9\xe8\xa7\x92\n/home/chenyihao/miniconda3/envs/vesselfm/bin/python scripts/split_branch_segments.py \\\n  --features outputs/Normal_1_features \\\n  --out outputs/Normal_1_segments \\\n  --segment-length 10 \\\n  --min-segment-length 2\n'' returned non-zero exit status 2.

In [ ]:
import json, pprint, pathlib
summary = json.loads(pathlib.Path('outputs/Normal_1_segments/segments_summary.json').read_text())
print('分支段数:', summary.get('segment_count'))
pprint.pp(summary['segments'][:5])

## 2. RCF 推理（PNG → NIfTI）
- 假设已有 RCF 训练好的权重 `results/RCF/ASOCA2020/checkpoint_epoch20.pth`。
- 若需重新切片可用 `scripts/prepare_rcf_asoca.py`，此处直接演示推理 + 拼成 NIfTI。


In [ ]:
%%bash
set -e
cd third_party/RCF-PyTorch
# 对 Normal_1 的 PNG 目录做推理；根据实际存放修改 --input-dir
python Ktest.py \
  --checkpoint results/RCF/ASOCA2020/checkpoint_epoch20.pth \
  --input-dir data/ASOCA2020_rcf/train/img \
  --save-dir results/RCF/ASOCA2020_predpng_Normal1

In [ ]:
%%bash
# 将预测 PNG 堆成与原 CTA 对齐的 NIfTI
python scripts/rcf_to_nifti.py \
  --rcf-dir third_party/RCF-PyTorch/results/RCF/ASOCA2020_predpng_Normal1 \
  --reference ASOCA2020/Normal/CTCA_nii/Normal_1.nii.gz \
  --output outputs/Normal_1_rcf_edges.nii.gz

### 可选：简化版推理（支持切换是否取反）
- 避免 Ktest 中的固定路径/参数，直接加载 checkpoint，逐切片推理 CTA 并生成 NIfTI。
- `invert=False` 时不再执行 `1 - fuse`，若发现预测与 GT 反相，可切换为 `True`。

In [ ]:
import sys, cv2, torch, nibabel as nib, numpy as np
from pathlib import Path
sys.path.insert(0, str(Path('third_party/RCF-PyTorch').resolve()))
from models import RCF

ckpt = Path('third_party/RCF-PyTorch/results/RCF/ASOCA2020/checkpoint_epoch10.pth')
ref_path = Path('ASOCA2020/Normal/CTCA_nii/Normal_1.nii.gz')
out_path = Path('outputs/Normal_1_rcf_edges_simplified.nii.gz')
invert = False  # 如需反相切到 True

img = nib.load(str(ref_path))
vol = img.get_fdata().astype(np.float32)
mean = np.array([104.00698793, 116.66876762, 122.67891434], dtype=np.float32)

model = RCF().cuda()
state = torch.load(str(ckpt), map_location='cuda', weights_only=False)
state_dict = state.get('state_dict', state)
model.load_state_dict(state_dict, strict=False)
model.eval()

edges = np.zeros_like(vol, dtype=np.float32)
with torch.no_grad():
    for z in range(vol.shape[0]):
        slice_arr = vol[z]
        vmax = float(np.max(slice_arr))
        slice_norm = cv2.convertScaleAbs(slice_arr, alpha=(255.0 / max(1e-3, vmax)))
        img_rgb = cv2.cvtColor(slice_norm, cv2.COLOR_GRAY2RGB).astype(np.float32)
        img_rgb = (img_rgb - mean).transpose(2,0,1)[None]
        inp = torch.from_numpy(img_rgb).cuda()
        fuse = torch.squeeze(model(inp)[-1]).detach().cpu().numpy().astype(np.float32)
        edges[z] = (1.0 - fuse) if invert else fuse

nib.save(nib.Nifti1Image(edges, img.affine, img.header), str(out_path))
out_path

PosixPath('outputs/Normal_1_rcf_edges_simplified.nii.gz')

## 3. 可选：重建 3D 网格或与 FGPM 融合
- 重建网格：`python -m vessel_seg.shape reconstruct --features outputs/Normal_1_features --output outputs/Normal_1.vtp`
- 在 FGPM 推理中使用边缘先验：`python -m vessel_seg.fgpm infer --edge-map outputs/Normal_1_rcf_edges.nii.gz ...`

以上 notebook 单元可按需修改路径/参数后依次运行。